In [ ]:
#import library 
import numpy as np 
import pandas as pd 
from sklearn.metrics import roc_curve,auc
from sklearn.model_selection import train_test_split #used for dividing the data into training and testing batches.

In [ ]:
df=pd.read_csv('pd-v2.1.csv')
df.columns

In [ ]:
#vectorizing the input ie, converting to numbers
df['HbA1c']=df['HbA1c'].map({
    '>7g%':2,
    'normoglycemic':0,
    '<7g%':1
})

In [ ]:
df['NUMBER OF CIGGARETTES']=df['NUMBER OF CIGGARETTES'].map({
    '0':0,
    '>10':2,
    '<10':1
})

In [ ]:
df['% OF BONE LOSS/Age ']=df['% OF BONE LOSS/Age '].map({
    '<1':0,
    '0.25-1':1,
    '>1':2
})

df['EXTENT AND DISTRIBUTION']=df['EXTENT AND DISTRIBUTION'].map({
    '<30%':0,
    '>30%':1
})

df['BLEEDING ON PROBING']=df['BLEEDING ON PROBING'].map({
    '<10%':0,
    '>10%':1
})
df['DIAGNOSIS']=df['DIAGNOSIS'].map({
    'Unstable Generalized Stage III Grade B Periodontitis':0,
    'Unstable Generalized Stage III Grade C Periodontitis':1,
    'Unstable Generalized Stage IV Grade B Periodontitis':2,
    'Unstable Generalized Stage IV Grade C Periodontitis':3,
    'Unstable Localized Stage II Grade B Periodontitis':4,
    'Unstable Localized Stage III Grade B Periodontitis':5,
    'Unstable Localized Stage III Grade C Periodontitis':6,
    'Unstable Localized Stage IV Grade B Periodontitis':7,
    'Unstable Localized Stage IV Grade C Periodontitis':8,
    'Unstable Generalized Stage IV Grade A Periodontitis':9
})

In [ ]:
df[:10]

In [ ]:
df.info()

In [ ]:
df[0:2]

In [ ]:
#target attribute
target=df['DIAGNOSIS']
target = target.reset_index(drop=True)
print(target.shape)

df=df.drop(['DIAGNOSIS'],axis=1)

In [ ]:
from lazypredict.Supervised import LazyClassifier

X_train, X_test, y_train, y_test = train_test_split(df, target,test_size=.5,random_state =123)

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(df,target,train_size=0.8,stratify=target)

from sklearn.svm import SVC
clf=SVC()
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

y_pred = clf.predict(X_test)
print("Accuracy score of the model is:",accuracy_score(y_test,y_pred)*100,"%")
print(classification_report(y_test, y_pred))


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(df,target,train_size=0.8,stratify=target)

from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
clf = Pipeline([                   
     ('DecissionTree',SVC())          
])
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_class = np.argmax(y_test, axis=0) 
y_pred_prob=clf.predict_proba(X_test)
print(y_pred.shape)

print("Accuracy score of the model is:",accuracy_score(y_test,y_pred)*100,"%")
print(classification_report(y_test, y_pred))
      
cm = confusion_matrix(y_test, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=clf.classes_)
disp.plot()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
import numpy as np


# y_test is the true labels of the test instances
# y_pred_prob is the predicted probabilities for each class
n_classes = len(np.unique(y_test))
y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot the ROC curves for each class
plt.figure(figsize=(8,6))
colors = cycle(['blue', 'red', 'green', 'purple', 'orange'])
su=0
for i, color in zip(range(n_classes), colors):
    su+=roc_auc[i]
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (AUC = {1:0.2f})'
             ''.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=2) # random guessing
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve for multi-class classification')
plt.legend(loc="lower right")
plt.show()
print('AUC',su/9)